<a href="https://colab.research.google.com/github/Gan4x4/CV-HSE2019/blob/master/video/Perfomance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Detector
!git clone https://github.com/ultralytics/yolov3.git

#MOT
!git clone https://github.com/abewley/sort.git

#Download video
!sudo wget https://yt-dl.org/downloads/latest/youtube-dl
!sudo chmod a+rx youtube-dl
!./youtube-dl -f22 -o video.mp4 https://www.youtube.com/watch?v=2RQrKP9a0XE

!pip install GPUtil

Cloning into 'yolov3'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 8723 (delta 45), reused 48 (delta 23), pack-reused 8645
Receiving objects: 100% (8723/8723), 6.95 MiB | 13.28 MiB/s, done.
Resolving deltas: 100% (6012/6012), done.
Cloning into 'sort'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 173 (delta 10), reused 32 (delta 6), pack-reused 136
Receiving objects: 100% (173/173), 1.20 MiB | 3.03 MiB/s, done.
Resolving deltas: 100% (56/56), done.
--2020-04-09 16:29:39--  https://yt-dl.org/downloads/latest/youtube-dl
Resolving yt-dl.org (yt-dl.org)... 95.143.172.170, 2001:1a50:11:0:5f:8f:acaa:177
Connecting to yt-dl.org (yt-dl.org)|95.143.172.170|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://yt-dl.org/downloads/2020.03.24/youtube-dl [fol

In [2]:
import humanize
import GPUtil as GPU
import psutil
import os
import torch

def time_synchronized():
    torch.cuda.synchronize() if torch.cuda.is_available() else None
    return time.time()

def gpu_usage():
    GPUs = GPU.getGPUs()
    # XXX: only one GPU on Colab and isn’t guaranteed
    if len(GPUs) == 0:
        return False
    gpu = GPUs[0]
    process = psutil.Process(os.getpid())
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

!nvidia-smi

Thu Apr  9 16:30:40 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
# Init model
import sys
sys.path.insert(0,'/content/yolov3')

from models import *
from utils.utils import *

device = torch.device("cuda:0")

# Default
input_size = (416, 416)
input_size = (256, 416)  # S4

model = Darknet("yolov3/cfg/yolov3.cfg", input_size ).to(device)

# Load weights
weights = 'yolov3/weights/yolov3.pt'
attempt_download(weights)
model.load_state_dict(torch.load(weights, map_location=device)['model'])

# Half presicion
model.half() #S2
model.eval()


Model Summary: 222 layers, 6.19491e+07 parameters, 6.19491e+07 gradients


Darknet(
  (module_list): ModuleList(
    (0): Sequential(
      (Conv2d): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (BatchNorm2d): BatchNorm2d(32, eps=0.0001, momentum=0.03, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (1): Sequential(
      (Conv2d): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (BatchNorm2d): BatchNorm2d(64, eps=0.0001, momentum=0.03, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (2): Sequential(
      (Conv2d): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (BatchNorm2d): BatchNorm2d(32, eps=0.0001, momentum=0.03, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (3): Sequential(
      (Conv2d): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


In [0]:
import queue
import cv2
import torch
from torch.utils.data import DataLoader
from torch.utils.data import IterableDataset
from torchvision import models, datasets, transforms
from PIL import Image
import csv

class VideoDataset(IterableDataset):
    def __init__(self, filename, input_size):
        super().__init__()
        self.stream = cv2.VideoCapture(filename)

    def read_next_frame(self):
        ret, frame = self.stream.read()
        while ret:
          img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
          img = cv2.resize(img,input_size)
          yield img
          ret, frame = self.stream.read()
        self.stream.release()
        return None

    def __iter__(self):
        return self.read_next_frame()

In [0]:
import cv2
from torch.utils.data import DataLoader
from torchvision import transforms
import time

dataset = VideoDataset("video.mp4",input_size)
batch_size = 1
batch_size = 64 #S1
batch_size = 128 #S3
dataloader = DataLoader(dataset,  batch_size=batch_size)

MAX_FRAMES = 300
frames_read = 0
tm = time_synchronized()
cnn_tm = 0
nms_tm = 0
base_log = max(50,batch_size)
log_interval = base_log - (base_log % batch_size)

!nvidia-smi

# TODO Add motion detector
for images in dataloader:
  frames_read += images.shape[0]
  with torch.no_grad():
    
    images.requires_grad = False
    images.to(device)
    images = images.type(torch.cuda.FloatTensor)
    images = images/255 # Now conversion on GPU
    images = images.permute(0,3, 1, 2)
    # Half
    images = images.half() #S2
    
    t1 = time_synchronized()
    output = model(images)
    detections = output[0]
    # Half
    detections = output[0].float() #S2
    t2 = time_synchronized()

    cnn_tm += t2-t1
    results = non_max_suppression(detections)

    t3 =time_synchronized()
    nms_tm += t3 - t2
  
  if not frames_read % log_interval :
    print (gpu_usage())
    print ("Processed {} frames. TOTAL_FPS: {:.1f}, YOLO_FPS: {:.1f}, NMS_FPS: {:.1f}" .format(frames_read,log_interval/(time_synchronized()-tm),log_interval/cnn_tm,log_interval/nms_tm))
    tm = time_synchronized()
    cnn_tm = 0
    nms_tm = 0
  if frames_read > MAX_FRAMES:
    break



Mon Mar 16 14:34:07 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.59       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    72W / 149W |   9814MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  